# Milestone 2

In [1]:
URL="http://10.128.0.4:8080"

In [2]:
!trino $URL --execute "show catalogs"

"bigquery"
"mongo"
"mysql"
"postgres"
"system"
"tpch"


### Q1

In [3]:
!trino $URL --execute "\
SELECT s.cust_id, s.email, r_s.count \
FROM mysql.final_project.shopper s \
JOIN postgres.final_project.reservations r \
    ON s.cust_id = r.cust_id \
JOIN bigquery.final_project.currency c \
    ON r.cnt_code = c.cntry_code \
JOIN (\
    SELECT s.cust_id, count(*) as count \
    FROM mysql.final_project.shopper s \
    JOIN postgres.final_project.reservations r \
    ON s.cust_id = r.cust_id \
    GROUP BY s.cust_id, s.email, s.cust_id \
) r_s \
    ON r_s.cust_id = s.cust_id \
WHERE c.curr_code = 'USD' AND r.city != s.city\
ORDER BY s.cust_id \
LIMIT 5"

"27","andersonsean@example.net","2"


### Q2

In [4]:
!trino $URL --execute "\
SELECT\
    s.cust_id,\
    s.email,\
    t.dep_airport,\
    t.dep_date\
FROM\
    mysql.final_project.shopper s\
JOIN\
    mongo.final_project.ticketing t ON s.cust_id = t.cust_id\
WHERE\
    t.airline = 'American Airlines'\
    AND t.stops = 0\
    AND t.dep_date BETWEEN DATE '2020-01-01' AND DATE '2024-12-31'\
ORDER BY\
    t.dep_date\
LIMIT 5"

"10","wgarcia@example.com","Wenzhou airport","2023-06-13 00:00:00.000"
"36","vazquezsarah@example.net","Chou Shui Tzu airport","2023-12-01 00:00:00.000"
"24","weavermichael@example.com","Trelew Almirante Zar airport","2023-12-05 00:00:00.000"


### Q3

In [5]:
!trino $URL --execute "\
SELECT\
    s.cust_id,\
    s.email,\
    SUM(r.pmt_amt) AS pmt_amt,\
    SUM(t.tik_amt) AS tik_amt,\
    SUM(r.pmt_amt + t.tik_amt) AS trip_amount\
FROM\
    mysql.final_project.shopper s\
JOIN\
    postgres.final_project.reservations r ON s.cust_id = r.cust_id\
JOIN\
    mongo.final_project.ticketing t ON s.cust_id = t.cust_id \
WHERE\
    r.arr_date = DATE(t.dep_date) \
    OR r.dep_date = DATE(t.arr_date) \
GROUP BY\
    s.cust_id, s.email\
ORDER BY\
    trip_amount DESC\
LIMIT 5"

"38","john03@example.com","410.17","4957.39","5367.56"
"41","jalvarado@example.org","909.66","3897.61","4807.27"
"1","diane45@example.org","191.72","716.76","908.48"


## Part 3

In [6]:
# Unlike postgres, trino does not have the types float or numeric. Instead, trino uses double for float deciaml for numeric. In addition, primary keys do not exist in trino

In [7]:
!trino $URL --execute "DROP TABLE IF EXISTS postgres.final_project.test"
!trino $URL --execute "\
    CREATE TABLE postgres.final_project.test (\
    id INT NOT NULL,\
    str1 VARCHAR(50) NOT NULL,\
    str2 CHAR(5) NOT NULL,\
    long FLOAT NOT NULL,\
    day DATE NOT NULL,\
    cost NUMERIC(5,2) NOT NULL,\
    PRIMARY KEY (id))"

!trino $URL --execute "\
    CREATE TABLE postgres.final_project.test (\
    id INT NOT NULL,\
    str1 VARCHAR(50) NOT NULL,\
    str2 CHAR(5) NOT NULL,\
    long DOUBLE NOT NULL,\
    day DATE NOT NULL,\
    cost DECIMAL(5,2) NOT NULL)"

!trino $URL --execute "\
    CREATE TABLE mongo.final_project.test (\
    id INT,\
    str1 VARCHAR(50),\
    str2 CHAR(5),\
    long DOUBLE,\
    day DATE,\
    cost DECIMAL(5,2))"

DROP TABLE
Query 20231207_002401_00005_mzkic failed: line 1:127: Unknown type 'FLOAT' for column 'long'
CREATE TABLE postgres.final_project.test (     id INT NOT NULL,     str1 VARCHAR(50) NOT NULL,     str2 CHAR(5) NOT NULL,     long FLOAT NOT NULL,     day DATE NOT NULL,     cost NUMERIC(5,2) NOT NULL,     PRIMARY KEY (id))

CREATE TABLE
Query 20231207_002405_00007_mzkic failed: line 1:1: Table 'mongo.final_project.test' already exists
CREATE TABLE mongo.final_project.test (     id INT,     str1 VARCHAR(50),     str2 CHAR(5),     long DOUBLE,     day DATE,     cost DECIMAL(5,2))



In [8]:
# Correct counts aquired for each database

In [9]:
!trino $URL --execute "select count(*) from mysql.final_project.shopper"
!trino $URL --execute "select count(*) from postgres.final_project.reservations"
!trino $URL --execute "select count(*) from mongo.final_project.ticketing"
!trino $URL --execute "select count(*) from bigquery.final_project.currency"

"50"
"100"
"100"
"30"


In [10]:
# Inserts were supported in all databases, Updates where only supported in postgres, and deletes worked for postgres and mongo.

In [11]:
# Inserts
!trino $URL --execute "insert into mysql.college.instructor values ('abc1234', 'John Smith', 'Mathematics')"
!trino $URL --execute "insert into postgres.college.instructor values ('abc1234', 'John Smith', 'Mathematics')"
!trino $URL --execute "insert into mongo.college.takes values ('abc1234', 'CS1234')"
!trino $URL --execute "insert into bigquery.college.instructor values ('abc1234', 'John Smith', 'Mathematics')"

# Updates
!trino $URL --execute "update mysql.college.instructor set name = 'Mary Jane' where tid = 'abc1234'"
!trino $URL --execute "update postgres.college.instructor set name = 'Mary Jane' where tid = 'abc1234'"
!trino $URL --execute "update mongo.college.takes set cno = 'CS9876' where sid = 'abc1234'"
!trino $URL --execute "update bigquery.college.instructor set name = 'Mary Jane' where tid = 'abc1234'"

# Deletes
!trino $URL --execute "delete from mysql.college.instructor where tid = 'abc1234'"
!trino $URL --execute "delete from postgres.college.instructor where tid = 'abc1234'"
!trino $URL --execute "delete from mongo.college.takes where sid = 'abc1234'"
!trino $URL --execute "delete from bigquery.college.instructor where tid = 'abc1234'"

Query 20231207_002416_00012_mzkic failed: Duplicate entry 'abc1234' for key 'Instructor.PRIMARY'

INSERT: 1 row
INSERT: 1 row
INSERT: 1 row
Query 20231207_002429_00016_mzkic failed: This connector does not support modifying table rows

UPDATE: 1 row
Query 20231207_002433_00018_mzkic failed: This connector does not support modifying table rows

Query 20231207_002434_00019_mzkic failed: This connector does not support modifying table rows

Query 20231207_002437_00020_mzkic failed: This connector does not support modifying table rows

DELETE: 1 row
DELETE: 1 row
Query 20231207_002443_00023_mzkic failed: This connector does not support modifying table rows



In [12]:
# In trino, transactions are not supported for writes 

In [13]:
!trino $URL --execute "start transaction;\
insert into mysql.college.class values ('abd1234     ', 'CS1234', 'Computer Class', 1, '  ');\
commit;"

!trino $URL --execute "start transaction;\
delete from postgres.college.teaches where cno = 'CS329E';\
commit;"

!trino $URL --execute "start transaction;\
delete from mongo.college.takes where cno = 'CS329E';\
commit;"

!trino $URL --execute "start transaction;\
insert into bigquery.college.instructor values ('abc1234', 'John Smith', 'Mathematics');\
commit;"


START TRANSACTION
Query 20231207_002445_00025_mzkic failed: Catalog only supports writes using autocommit: mysql

START TRANSACTION
Query 20231207_002447_00027_mzkic failed: Catalog only supports writes using autocommit: postgres

START TRANSACTION
Query 20231207_002449_00029_mzkic failed: Catalog only supports writes using autocommit: mongo

START TRANSACTION
Query 20231207_002451_00031_mzkic failed: Catalog only supports writes using autocommit: bigquery



In [14]:
!trino $URL --execute "start transaction;select count(*) from mysql.final_project.shopper;commit"

START TRANSACTION
"50"
COMMIT


## Trino Evaluation Summary

        In order to test DDL, we tried dropping and creating tables. We found that trino does not support primary keys. The CREATE TABLE statement with 'PRIMARY KEY (id)' against Postgres CREATE TABLE postgres.final_project.test resulted in an error. Trino also does not support float or numeric types. Instead, in our create table statement, we had to specify float or numeric types as 'DECIMAL' or 'DOUBLE'. Trino does support drops: running DROP TABLE IF EXISTS postgres.final_project.test successfully resulted in the table being dropped.
        In order to test DML, we tried inserts, updates, and deletes. We ran INSERT INTO database.table.column VALUES for each database, and found that trino allows for inserts across all 4 databases. We ran UPDATE database.table.column SET for each database, and found that trino only allows updates for postgres. The other 3 databases resulted in an error: "This connector does not support modifying table rows". Lastly, we ran DELETE FROM database.table.column for each database, and found that trino only allows deletes for postgres and mongo. Similar to updates, mysql and bigquery resulted in an error: "This connector does not support modifying table rows".
        In addition, we also tried running transcations to further test DML. We found that trino does not support for transcations for writes across all 4 databases. We ran START TRANSACTION and COMMIT for inserts/deletes (depending on what trino allows for the database explained above), but always resulted in an error: "Catalog only supports writes using autocommit". However, trino does support transactions for SELECT statements. We ran START TRANSACTION; SELECT COUNT(*) FROM mysql.final_project.shopper;COMMIT, and it correctly resulted in a count of 50.
        Lastly, we tested SELECT statements by collecting the counts of the final_project table in each database. We found that trino supports SELECT statements across all 4 databases. For example, when we ran SELECT COUNT(*) FROM mysql.final_project.shopper, the query correctly resulted in a count of 50. Similarly, it obtained the counts correctly for the other 3 databases.